# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [3]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
import os
from glob import glob

# Write your code below.
PRICE_DATA=os.getenv("PRICE_DATA")
parquet_files=glob(os.path.join("../",PRICE_DATA, "*/*/*.parquet")) #additional directory level up compared to original env path (meant for materials directory)

In [6]:
parquet_files[:3]

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
#dd_px.compute().head()

In [8]:
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532215,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206852,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712803,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276855,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299919,2819626,Health Care,Life Sciences Tools & Services,2000


In [9]:
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

In [10]:
# Write your code below.
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] -x['Low']
))



C:\Users\ShwetankPC\AppData\Local\Temp\ipykernel_91308\988898579.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [11]:
# Write your code below.

pd_feat=dd_feat.compute()

In [15]:
pd_feat=pd_feat.reset_index()
pd_feat['Rolling Average']=pd_feat.groupby(['ticker'], as_index=False)[['returns']].rolling(10).mean()['returns']

In [38]:
pd_feat.head()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,Rolling Average
0,IP,2006-01-03,31.789791,31.789791,30.921268,31.276150,15.694667,2985132,Materials,Paper & Plastic Packaging Products & Materials,2006,NaN,NaN,0.868523,NaN
1,IP,2006-01-04,31.229454,31.546980,31.192099,31.518963,15.816516,2222948,Materials,Paper & Plastic Packaging Products & Materials,2006,31.276150,0.007764,0.354881,NaN
2,IP,2006-01-05,31.350861,31.416233,31.192099,31.416233,15.764959,1702547,Materials,Paper & Plastic Packaging Products & Materials,2006,31.518963,-0.003259,0.224134,NaN
3,IP,2006-01-06,31.565657,31.985909,31.378878,31.920538,16.018028,2205173,Materials,Paper & Plastic Packaging Products & Materials,2006,31.416233,0.016052,0.607031,NaN
4,IP,2006-01-09,32.191364,32.191364,31.462929,31.668385,15.891499,2574593,Materials,Paper & Plastic Packaging Products & Materials,2006,31.920538,-0.007899,0.728436,NaN


In [39]:
pd_feat.tail()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,Rolling Average
2743689,BXP,2003-12-24,48.000000,48.360001,47.959999,48.340000,19.521795,145100,Real Estate,Office REITs,2003,47.980000,0.007503,0.400002,0.003688
2743690,BXP,2003-12-26,47.980000,48.049999,47.680000,47.950001,19.619993,120600,Real Estate,Office REITs,2003,48.340000,-0.008068,0.369999,0.001379
2743691,BXP,2003-12-29,47.980000,48.230000,47.849998,48.160000,19.705923,145100,Real Estate,Office REITs,2003,47.950001,0.004380,0.380001,0.002134
2743692,BXP,2003-12-30,48.200001,48.400002,48.160000,48.230000,19.734556,198500,Real Estate,Office REITs,2003,48.160000,0.001453,0.240002,0.001961
2743693,BXP,2003-12-31,48.250000,48.470001,48.099998,48.189999,19.718204,182000,Real Estate,Office REITs,2003,48.230000,-0.000829,0.370003,0.001223


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? \
Ans: The rolling method does exist in dask so it was not.necessary

+ Would it have been better to do it in Dask? Why? \
Ans: It would have probably been better to do it in Dask as it is meant to work with the large amount of data present and would be more efficient (also saves time on conversion to pandas)

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.